# convect each results into excel

In [ ]:
import pandas as pd

df = pd.read_csv('results/summarized_results_qt1.csv')

# Sort the dataframe

df_sorted = df.drop(columns=['nn_model', 'number_of_clients'])
# Sort the dataframe by skew, dataset, and heterogeneity_class

# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
# Add a newline before each occurrence of 'fedavg' in the 'exp_type' column
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
# Sort the dataframe by the defined order
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('results.xlsx', index=False)

In [5]:
df=pd.read_csv('granular_results/noqs.csv')
# Define the order and bold the 'cornflqs' exp_type
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('noqs.xlsx', index=False)

In [7]:
df=pd.read_csv('/home/mbenali/Documents/Project/CFL-Evaluation-of-labels-and-quantity-skew/granular_results/qs1.csv')
df_sorted = df
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs1.xlsx', index=False)


In [6]:
df=pd.read_csv('granular_results/qs2.csv')
# Define the order and bold the 'cornflqs' exp_type
exp_type_order = ['fedavg', 'fedprox', 'cfl', 'hcfl', 'fedgroup', 'ifca', 'srfca', 'cornflqs', 'oracle-centralized']
df_sorted = df
df_sorted['exp_type'] = pd.Categorical(df_sorted['exp_type'], categories=exp_type_order, ordered=True)
df_sorted['exp_type'] = df_sorted['exp_type'].apply(lambda x: '\n' + x if x == 'fedavg' else x)
df_sorted = df_sorted.sort_values(['skew', 'dataset', 'heterogeneity_class','exp_type'])
df_sorted.to_excel('qs2.xlsx', index=False)

# calculate ranking 

In [ ]:
def algo_ranking(dataset=None, skew=None, heterogeneity_class=None):
    import pandas as pd
    # Load the CSV data into a pandas DataFrame
    df = pd.read_csv('results/summarized_results.csv')
    if skew == 'quantity-skew-type-1' : 
        df = pd.read_csv('granular_results/qs1.csv')
        df.rename(columns={'global_accuracy': 'accuracy', 'homogeneity': 'hom', 'completeness': 'cmplt', 'v_measure': 'vm'}, inplace=True)
        granular_metrics = ['class_1_qt-skew_0.05', 'class_2_qt-skew_0.05', 'class_3_qt-skew_0.05', 'class_4_qt-skew_0.05',
                         'class_1_qt-skew_0.2', 'class_2_qt-skew_0.2', 'class_3_qt-skew_0.2', 'class_4_qt-skew_0.2',
                         'class_1_qt-skew_1', 'class_2_qt-skew_1', 'class_3_qt-skew_1', 'class_4_qt-skew_1',
                         'class_1_qt-skew_2', 'class_2_qt-skew_2', 'class_3_qt-skew_2', 'class_4_qt-skew_2']
        for metric in granular_metrics:
            df[metric] = df[metric].str.split(' ').str[0].astype(float)
        
    elif skew == 'quantity-skew-type-2' : 
        df = pd.read_csv('granular_results/qs2.csv')
        df.rename(columns={'global_accuracy': 'accuracy', 'homogeneity': 'hom', 'completeness': 'cmplt', 'v_measure': 'vm'}, inplace=True)
        granular_metrics= ['class_1_qt-skew_0.05', 'class_4_qt-skew_0.2', 'class_2_qt-skew_1', 'class_3_qt-skew_2']
        
        for metric in granular_metrics:
            df[metric] = df[metric].str.split(' ').str[0].astype(float)
     
    metrics = ['accuracy', 'ARI', 'AMI', 'hom', 'cmplt', 'vm']
    
    df.fillna('None', inplace=True)
    df = df[df['exp_type'] != 'oracle-centralized']
    df['exp_type'] = df['exp_type'] + '_' + df['params']
    
    # Filter the dataframe based on the input parameters
    if dataset:
        df = df[df['dataset'] == dataset]
    if skew:
        if skew == 'None':
            df = df[df['skew'] == skew]
        elif skew == 'quantity-skew':
            df = df[df['skew'] != 'None']
        else:
            df = df[df['skew'] == skew]
    if heterogeneity_class:
        df = df[df['heterogeneity_class'] == heterogeneity_class]

    # Step 2: Clean the data
    # Remove the '\pm' and uncertainty values from the 'accuracy' column
    df['accuracy'] = df['accuracy'].str.split(' ').str[0].astype(float)
    # Replace 'n/a' with NaN for missing values

    # Convert metric columns to numeric

    
    df[metrics] = df[metrics].apply(pd.to_numeric, errors='coerce')
    
    # Concatenate 'dataset', 'heterogeneity_class', and 'skew' into one column named 'group'
    df['group'] = df['dataset'] + '_' + df['heterogeneity_class'] + '_' + df['skew'] + '_' + df['seed'].astype(str)
    # Drop the original 'dataset', 'heterogeneity_class', and 'skew' columns
    df = df.drop(columns=['dataset', 'heterogeneity_class', 'skew', 'seed'])

    # Step 3: Group the data by the new 'group' column
    grouped = df.groupby('group')

    # Step 4: Rank algorithms within each group for each metric
    def rank_algorithms(group):
        for metric in metrics:
            group[f'{metric}_rank'] = group[metric].rank(ascending=False, method='min')
        return group

    ranked_groups = grouped.apply(rank_algorithms)
    best = ranked_groups.groupby('exp_type')[[f'{metric}_rank' for metric in metrics]].mean()
    best = best.sort_values(by='accuracy_rank')

    # Calculate the average accuracy for each exp_type
    avg_accuracy = ranked_groups.groupby('exp_type')['accuracy'].mean()
    std_accuracy = ranked_groups.groupby('exp_type')['accuracy'].std()
    best['avg_accuracy'] = avg_accuracy
    best['std_accuracy'] = std_accuracy
    avg_ARI = ranked_groups.groupby('exp_type')['ARI'].mean()
    best['avg_ARI'] = avg_ARI
    # Calculate the average for each granular metric for each exp_type
    if skew in ['quantity-skew-type-1', 'quantity-skew-type-2']:
        for metric in granular_metrics:
            avg_metric = ranked_groups.groupby('exp_type')[metric].mean()
            best[f'{metric}'] = avg_metric
            # Reorder the columns
    best = best[['accuracy_rank', 'avg_accuracy','std_accuracy', 'ARI_rank', 'avg_ARI', 'AMI_rank', 'hom_rank', 'cmplt_rank', 'vm_rank']]
    return best


In [25]:
from openpyxl.styles import Font

result = algo_ranking()
results_qt1 = algo_ranking(skew='quantity-skew-type-1')
results_qst2 = algo_ranking(skew='quantity-skew-type-2')
results_noqs = algo_ranking(skew='None')

from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Border, Side

# Load the existing workbook
wb = load_workbook('Results.xlsx')

# Create a new sheet
ws = wb.create_sheet(title='Global_results')

# Function to write a dataframe to the worksheet with formatting
def write_dataframe_to_sheet(ws, df, start_row):
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
    for r_idx, row in enumerate(dataframe_to_rows(df.round(2), index=True, header=True), start_row):
        for c_idx, value in enumerate(row, 1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            cell.border = thin_border
            # Bold the first exp_type and underline the second
            if r_idx == start_row + 1:
                cell.font = Font(bold=True)
            elif r_idx == start_row + 2:
                cell.font = Font(bold=True)    
            elif r_idx == start_row + 3:
                cell.font = Font(underline='single')
    return r_idx + 2  # Add an extra line between tables

# Write the first title and results
ws.append(['Global_ranking'])
ws.cell(row=ws.max_row, column=1).font = Font(bold=True)
start_row = write_dataframe_to_sheet(ws, result, ws.max_row + 1)

# Write the fourth title and results
ws.append(['No Quantity Skew'])
ws.cell(row=ws.max_row, column=1).font = Font(bold=True)
write_dataframe_to_sheet(ws, results_noqs, ws.max_row + 1)

# Write the second title and results
ws.append(['Quantity skew type 1'])
ws.cell(row=ws.max_row, column=1).font = Font(bold=True)
start_row = write_dataframe_to_sheet(ws, results_qt1, ws.max_row + 1)

# Write the third title and results
ws.append(['Quantity skew type 2'])
ws.cell(row=ws.max_row, column=1).font = Font(bold=True)
start_row = write_dataframe_to_sheet(ws, results_qst2, ws.max_row + 1)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    column = col[0].column_letter  # Get the column name
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = (max_length + 2)
    ws.column_dimensions[column].width = adjusted_width

# Save the workbook
wb.save('Results.xlsx')


In [2]:
from utils_results import granular_results
granular_results("results/")

In [3]:
from utils_results import main_excel
main_excel()

Excel file created successfully!


# EXP CSV generation

In [ ]:
import pandas as pd
import itertools

# Define the list of tuples for exp_type and params
exp_params_list = [
    ('fedavg', 'None'),
    ('fedprox', 0.1),
    ('cfl', 'None'),
    ('hcfl', 'euclidean'),
    ('fedgroup', 'edc'),
    ('ifca', 'best'),
    ('srfca', 'None'),
    ('cornflqs', '1-10-10'),
    ('cornflqs', '10-1-10'),
    ('cornflqs', '1-1-20'),
    ('oracle-centralized', 'clustering')
]

# Define the base data
base_data = [
    ['mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['fashion-mnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['fashion-mnist', 'features-distribution-skew', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-features', 'quantity-skew-type-2'],
    ['kmnist', 'concept-shift-on-labels', 'quantity-skew-type-2'],
    ['kmnist', 'features-distribution-skew', 'quantity-skew-type-2']
]

# Define fixed parameters
nn_model = 'linear'
number_of_clients = 100
num_samples_by_label = 50
num_clusters = 4
default_epochs = 10
rounds = 20
# Define the seed values
seed_values = [42, 43, 44, 45, 46]

# Create a list to store all the combinations
all_combinations = []

# Generate all combinations of base_data, exp_params_list, and seed_values
for base, (exp_type, params), seed in itertools.product(base_data, exp_params_list, seed_values):
    epochs = 100 if exp_type == 'oracle-centralized' else default_epochs
    combination = base + [exp_type, params, seed, nn_model, number_of_clients, num_samples_by_label, num_clusters, epochs, rounds]
    all_combinations.append(combination)

# Convert the list of combinations to a DataFrame
columns = ['dataset', 'heterogeneity_class', 'skew', 'exp_type', 'params', 'seed', 'nn_model', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds']
df_combinations = pd.DataFrame(all_combinations, columns=columns)

# Reorder the columns as specified
df_combinations = df_combinations[['exp_type', 'params', 'dataset', 'nn_model', 'heterogeneity_class', 'skew', 'number_of_clients', 'num_samples_by_label', 'num_clusters', 'epochs', 'rounds', 'seed']]

# Save the DataFrame to a CSV file
df_combinations.to_csv('exp_configs.csv', index=False)